In [1]:
import pandas as pd
import numpy as np
from cointegracao import Cointegration
from itertools import combinations, permutations
import time
from backtest import Executer
from tqdm.contrib.concurrent import process_map, thread_map # or thread_map
import gc

In [2]:
import execute

In [3]:
n_workers = 7 # Número de CPUS que vai usar, na minha máquina ficou bom com 8, começa testando com 1 e ve se vai melhorando
periodo = 'diario' # Período de teste pode ser 'diario' ou 'minutos'

# Período de tempo que vai olhar para tras para calcular a cointegração: 
# no diário uso 252, no de 15 minutos combinamos 3 meses = 15(minutos) * 21(dias uteis) * 3(meses) = 945
train_size = 252 
significancia = 0.05 # Significancia usada na cointegração
z_score_saida = 0.5 # Z-score que considera retorno a média, quando ele vai sair.

# Z-score de stop quando o resíduo está indo para longe do retorno a média:
# pode ser 'desligado' ou valores acima de 1
z_score_stop = 'desligado'
conf_var = 0.05 # Confiança do VAR. Também aceita 'desligado' para o caso sem stop

usar_halflife = 'desligado' # Usa o stop no halflife. aceita 'ligado' e 'desligado'
usar_var = 'ligado' # Usa o var como stop. Aceita 'ligado' e 'desligado'


In [4]:
if __name__ == '__main__':
    resultados_raw = execute.executa(
        n_workers = n_workers,
        periodo = periodo,
        train_size = train_size,
        significancia = significancia,
        z_score_saida = z_score_saida,
        z_score_stop = z_score_stop,
        conf_var = conf_var,
        usar_halflife = usar_halflife,
        usar_var = usar_var    
    )
    

Executando 2020-01-01/2020-07-01


  0%|          | 0/386 [00:00<?, ?it/s]

In [5]:
# Calcula o retorno a partir dos resultados gerados no backtest
df_returns = execute.calcula_retornos(resultados_raw)

# Salva um arquivo que preciso para fazer os gráficos e tabelas depois 
filename = f'{periodo}_{train_size}_{significancia}_{z_score_saida}_{z_score_stop}_{conf_var}_{usar_halflife}_{usar_var}.csv'
df_returns.to_csv(filename, index=False)

C:\Users\willi\Documents\GitHub\cointegration\execute.py:213: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df_returns = df_returns.set_index('open_date').loc['2019-01-01':]


In [6]:
pd.read_csv(filename)

,open_date,date,pair,status,price_fst_stock,price_scnd_stock,beta_open,beta_close,last_residual,std_residual,...,setor,sorted_pair,side,id,ratio_sem_custos,ratio,return,retorno_acumulado,return_sem_custos,retorno_acumulado_sem_custos
0,2020-04-16,2020-04-16,"('ABEV3', 'BRFS3')",True,10.856538,19.133183,0.270387,NaN,-2.051439,0.844314,...,Alimentos e Bebidas,"('ABEV3', 'BRFS3')",lower,1,0.559419,0.567419,NaN,NaN,NaN,NaN
1,2020-04-16,2020-04-17,"('ABEV3', 'BRFS3')",True,10.920000,19.050000,0.270387,NaN,-1.803740,0.848934,...,Alimentos e Bebidas,"('ABEV3', 'BRFS3')",lower,1,0.573228,0.573228,0.010238,0.010238,0.024686,0.024686
2,2020-04-16,2020-04-20,"('ABEV3', 'BRFS3')",True,10.930000,18.920000,0.270387,NaN,-1.709729,0.851268,...,Alimentos e Bebidas,"('ABEV3', 'BRFS3')",lower,1,0.577696,0.577696,0.007793,0.018110,0.007793,0.032671
3,2020-04-16,2020-04-22,"('ABEV3', 'BRFS3')",True,10.940000,19.450000,0.270387,NaN,-1.804413,0.856254,...,Alimentos e Bebidas,"('ABEV3', 'BRFS3')",lower,1,0.562468,0.562468,-0.026359,-0.008726,-0.026359,0.005450
4,2020-04-16,2020-04-23,"('ABEV3', 'BRFS3')",True,10.870000,19.510000,0.270387,NaN,-1.854607,0.863754,...,Alimentos e Bebidas,"('ABEV3', 'BRFS3')",lower,1,0.557150,0.557150,-0.009454,-0.018098,-0.009454,-0.004055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,2020-05-19,2020-05-19,"('MGLU3', 'AMER3')",True,14.713731,89.202136,0.161677,NaN,-1.711611,0.826413,...,Varejo,"('AMER3', 'MGLU3')",lower,95,0.162622,0.164948,NaN,NaN,NaN,NaN
786,2020-05-19,2020-05-20,"('MGLU3', 'AMER3')",True,14.190000,86.310000,0.161677,NaN,-1.509559,0.826899,...,Varejo,"('AMER3', 'MGLU3')",lower,95,0.164407,0.164407,-0.003279,-0.003279,0.010976,0.010976
787,2020-05-19,2020-05-21,"('MGLU3', 'AMER3')",True,14.530000,84.420000,0.161677,NaN,-0.834491,0.822162,...,Varejo,"('AMER3', 'MGLU3')",lower,95,0.172116,0.172116,0.046885,0.043453,0.046885,0.058376
788,2020-05-19,2020-05-22,"('MGLU3', 'AMER3')",True,14.380000,83.010000,0.161677,NaN,-0.732784,0.816808,...,Varejo,"('AMER3', 'MGLU3')",lower,95,0.173232,0.173232,0.006487,0.050222,0.006487,0.065241


In [7]:
# Transforma do jeito que vc gosta pra salvar em excel
df_returns.pivot(index='pair', columns='date', values='return').T

pair,"(ABEV3, BRFS3)","(AMER3, MGLU3)","(BBAS3, BBDC4)","(BBDC3, SANB11)","(BBSE3, SULA11)","(BPAC11, SULA11)","(BRAP4, GGBR4)","(BRAP4, GOAU4)","(BRFS3, ABEV3)","(BRFS3, JBSS3)",...,"(SANB11, BBDC3)","(SBSP3, ENEV3)","(SBSP3, EQTL3)","(SBSP3, TAEE11)","(SULA11, BBSE3)","(SUZB3, KLBN11)","(TAEE11, CPLE6)","(TAEE11, SBSP3)","(USIM5, VALE3)","(VALE3, USIM5)"
date,,,,,,,,,,,,,,,,,,,,,
2019-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.007201,NaN,NaN
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.017491,NaN,NaN
2020-01-06,NaN,NaN,NaN,NaN,0.009093,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.016667,NaN,NaN
2020-01-07,NaN,NaN,NaN,NaN,0.013943,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.008572,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-16,NaN,NaN,NaN,NaN,NaN,NaN,0.052849,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Salva como arquivo de excel. Lembra sempre de escolher um 'nome_arquivo.xlsx' diferente,
# senão ele vai salvar em cima do arquivo existente.

df_returns.pivot(index='pair', columns='date', values='return').T.to_excel('nome_arquivo.xlsx')

In [9]:
df_returns['return'].sum()

-0.7673183094205693